# Wide vs long table format

## Summary

This was evaluated on WHO indicators dataset with about 30k variables and 16k rows (in wide format).

* **Long format & parquet sucks, we should use wide & feather for maximum performance** (or DuckDB instead of feather if loading files from S3 is too slow)

* **The most straightfoward solution would be to split the dataset into ~10 tables and save them as wide tables**. Since we'd be indexing them anyway, we could get the path to variables easily. E.g. path would be `dataset_5438/variables_0_to_1000/age_group_years_5_to_17`.

* Alternative is to keep it wide and be ok with ~200ms latency (and about 3mins processing time), keeping it in long format in feather format still takes ~200ms!

* Querying large dataset would be slow for both feather and parquet formats (long doesn't have indexing and wide takes a long time because there are too many columns)

* Indexable DB (MySQL, SQLite) with data stored in long format would be optimal for quering single variable (<10ms performance). Loading data into DuckDB to disk and indexing doesn't help (same performance as loading from feather), we'd have to load it into memory and index it.

## Notes

There are tradeoffs between storing data in long or wide format. Choosing a format is a prerequisite for choosing the right tool from the solution space.

Most of our datasets would benefit from the wide tabular format, but there are some datasets that are almost impossible (or at least impractical) to store in a wide format without post-processing. This happens when data is too sparse (i.e. each variable only contains a few countries and years) when there are a lot of variables. Such an example are WHO indicators with 30k variables (a lot of them are dimensions) where a long format has 45MB while the wide format has almost 2GB uncompressed (with only about 1% non-null values). On the other hand, covid data benefit from wide format since we have data for almost every country & date.

### Wide format

- Most of our datasets benefit from converting to wide format
- Variables can have different types & be stored efficiently in feather file
- Querying single variable from a dataset is fast (<10ms unless the dataset has >1000 columns)
- Works very poorly for a few datasets

### Long format

- Universal format, but inefficient storage for most datasets
- Same as what we use in MySQL
- Fast queries from a database (with index), but queries from feather could be slow if the dataset is large

### Split long datasets into multiple tables

- Variables from these huge datasets could be grouped into tables to have smaller feather files and more efficient lookups
- Grouping can be either manual (very tedious) or automatic (table names wouldn’t make any sense which is not very catalog friendly)... alternatively we would reimplement these datasets in `ETL` (since most of them are from importers)
- Need for lookup table to identify to which table & column variable id belongs to

### Support both wide & long format

- Very efficient querying of most datasets (~10ms), but for instance querying a single variable from WHO indicators (stored in long feather format) could take 200ms anyway
- Querying large datasets in long format can be slow if we store it in feather format (as it doesn’t have index), but if we used some kind of database (DuckDB or Clickhouse) then maybe indexing could help a lot
- Need to manage both formats

In [1]:
# imports and notebook defaults
from nbinit import *

# Prepare datasets

In [2]:
from owid.catalog.utils import underscore
from owid.catalog.frames import repack_frame

# long format example
path = '/Users/mojmir/projects/etl/data/backport/owid/latest/dataset_5438_global_health_observatory__world_health_organization__2021_12/dataset_5438_global_health_observatory__world_health_organization__2021_12.feather'

# wide format example
# path = '/Users/mojmir/projects/etl/data/backport/owid/latest/dataset_5582_global_carbon_budget__global_carbon_project__v2021/dataset_5582_global_carbon_budget__global_carbon_project__v2021.feather'
# path = '/Users/mojmir/projects/etl/data/backport/owid/latest/dataset_5357_world_development_indicators__world_bank__2021_07_30/dataset_5357_world_development_indicators__world_bank__2021_07_30.feather'
lf = pd.read_feather(path)

# raw data is in long format
if 'variable_name' in lf.columns:
    lf = lf[['entity_name', 'variable_name', 'year', 'value']]

    # `value` is categorical for some reason, use `object` and fix it in repack_frame
    lf = lf.astype({'value': object})

    # use underscored variable names
    lf.variable_name = lf.variable_name.map(underscore)

    # convert to wide format
    wf = lf.pivot(index=['entity_name', 'year'], columns='variable_name', values='value')

    # resetting index and repacking can be very slow - 3 mins on my local
    wf = repack_frame(wf.reset_index())

# raw data is in wide format
else:
    wf = lf
    lf = wf.melt(id_vars=['entity_name', 'year'], value_vars=wf.columns[2:])
    lf = lf.astype({'entity_name': 'category', 'year': 'category', 'variable': 'category'}).rename(columns={'variable': 'variable_name'})

In [3]:
# feather and parquet use compression by default!
lf.to_feather('data/long.feather')
lf.to_parquet('data/long.parquet', index=False)
wf.to_feather('data/wide.feather')
wf.to_parquet('data/wide.parquet', index=False)

In [4]:
!ls -alh data/

total 454872
drwxr-xr-x  6 mojmir  staff   192B Apr 27 10:01 .
drwxr-xr-x  9 mojmir  staff   288B Apr 27 11:10 ..
-rw-r--r--  1 mojmir  staff    46M Apr 27 11:39 long.feather
-rw-r--r--  1 mojmir  staff    27M Apr 27 11:39 long.parquet
-rw-r--r--  1 mojmir  staff    75M Apr 27 11:39 wide.feather
-rw-r--r--  1 mojmir  staff    74M Apr 27 11:39 wide.parquet


In [5]:
print(f'Long format shape: {lf.shape}')
print(f'Wide format shape: {wf.shape}')
print(f'Long format memory: {lf.memory_usage(deep=True).sum() / 1e6:.2f} MB')
print(f'Wide format memory: {wf.memory_usage(deep=True).sum() / 1e6:.2f} MB')
print(f'Density of wide format: {wf.notnull().sum().sum() / wf.shape[0] / wf.shape[1]:.2%}')

Long format shape: (6117600, 4)
Wide format shape: (15933, 32100)
Long format memory: 423.19 MB
Wide format memory: 1834.73 MB
Density of wide format: 1.20%


## Reading latency of a single variable

In [6]:
import pyarrow.feather as feather
import pyarrow.parquet as parquet
import duckdb

variable = lf.variable_name.sample(1).iloc[0]
variable

'indicator__deaths_per_100_000_population__sex__female__age_group__30_49_years__causes__bipolar_disorder'

In [7]:
%%timeit -n5 -r1
# feather long format
t = feather.read_table('data/long.feather')
db = duckdb.arrow(t)
db.query('t', f"SELECT * FROM t WHERE variable_name = '{variable}'")

179 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 5 loops each)


In [8]:
%%timeit -n5 -r1
# feather wide format
t = feather.read_table('data/wide.feather', columns=['entity_name', 'year', variable])
db = duckdb.arrow(t)
db.query('t', f"SELECT * FROM t WHERE {variable} is not null")

146 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 5 loops each)


In [9]:
%%timeit -n5 -r1
# parquet long format with `filters`
t = parquet.read_table('data/long.parquet', filters=[('variable_name', '=', variable)])

738 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 5 loops each)


In [10]:
%%timeit -n5 -r1
# parquet long format with duckdb
t = parquet.read_table('data/long.parquet')
db = duckdb.arrow(t)
db.query('t', f"SELECT * FROM t WHERE variable_name = '{variable}'")

652 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 5 loops each)


In [11]:
%%timeit -n5 -r1
# parquet wide format
t = parquet.read_table('data/wide.parquet', columns=['entity_name', 'year', variable])
db = duckdb.arrow(t)
db.query('t', f"SELECT * FROM t WHERE {variable} is not null")

1.61 s ± 0 ns per loop (mean ± std. dev. of 1 run, 5 loops each)


In [12]:
# create database on disk, it would be way faster in memory, but we can't afford that
conn = duckdb.connect('db.duck')

In [13]:
%%timeit -n5 -r1
# parquet long format through DuckDB
conn.execute(f"select * from read_parquet('data/long.parquet') where variable_name = '{variable}'").fetch_arrow_table()

196 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 5 loops each)


In [14]:
%%timeit -n5 -r1
# parquet wide format with duckdb
conn.execute(f"select entity_name, year, {variable} from read_parquet('data/wide.parquet') where {variable} is not null").fetch_arrow_table()

451 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 5 loops each)


In [15]:
# add long format to DuckDB to allow indexing
conn.execute('DROP TABLE long');
conn.execute(f"CREATE TABLE long AS SELECT * FROM read_parquet('data/long.parquet');")

<_duckdb_extension.DuckDBPyConnection object at 0x13f418cf0>

In [16]:
%%timeit -n5 -r1
conn.execute(f"select * from long where variable_name = '{variable}'").fetch_arrow_table()

847 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 5 loops each)


In [17]:
# creating index doesn't help
conn.execute('CREATE INDEX variable_name_idx ON long (variable_name);')

<_duckdb_extension.DuckDBPyConnection object at 0x13f418cf0>

In [18]:
%%timeit -n5 -r1
conn.execute(f"select * from long where variable_name = '{variable}'").fetch_arrow_table()

504 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 5 loops each)


In [19]:
# loading 30k columns to duckdb takes forever
if wf.shape[1] <= 5000:
    conn.execute(f"CREATE TABLE wide AS SELECT * FROM read_parquet('data/wide.parquet');")

In [ ]:
%%timeit -n5 -r1
if wf.shape[1] <= 5000:
    # parquet wide format with duckdb
    conn.execute(f"select entity_name, year, {variable} from wide where {variable} is not null").fetch_arrow_table()

6.1 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 5 loops each)
